# Welcome to IMCity

Welcome to the IMCity challenge. You'll be trading products on our custom-built **CMI Exchange** whose settlements are driven by real-world London data — tidal levels at Westminster, weather conditions, and flight activity at Heathrow.

There are 8 products: 2 Thames tidal products, 2 weather products, 2 flight products, an ETF combining them, and an options structure on the ETF. Your job is to build a profitable trading strategy.

This notebook walks you through everything you need: connecting to the exchange, understanding the products and their settlement formulas, fetching data, and building an automated trading bot.

---

**What's inside:**

1. **Connect** — register and authenticate
2. **Products** — all 8 products with settlement formulas
3. **Orderbook** — understand the market
4. **Trading** — place GTC orders, cancel, IOC
5. **Live events** — react to orderbook and trade updates
6. **Strategy ideas** — approaches you might take
7. **Data sources** — helpers for weather, tides, and flights
8. **Price-time priority** — how the matching engine works
9. **Simple quoter** — a working market-making bot

## Rules

**Accounts & access:**
- One account per team on the challenge exchange. Each team is permitted a single running bot on the challenge exchange.
- The test exchange is open for experimentation — one account per person is permitted.

**Fair play:**
- We're here to be smart and competitive, but also to have fun. Make sure everyone has an equal chance.
- If we observe behaviour that would be illegal on a regulated market, we will ban the team. This includes:
  - **Spoofing** — placing orders you intend to cancel before they fill, to mislead others
  - **Layering** — stacking multiple orders on one side to create a false impression of supply/demand
  - **Wash trading** — colluding with another team to generate fake volume/profits
  - **Market manipulation** — any deliberate attempt to distort prices away from fair value to disadvantage others

**API usage:**
- Do not send more than **one request per second** (excluding the SSE market stream — that's free).
- Excessive request loops will be treated as abuse and can result in a ban.
- Do not attempt to exploit, overload, or break the exchange. Report bugs to the organisers.

## Prerequisites

Run the first code cell to install dependencies. Requires **Python 3.10+**.

In [ ]:
# Install dependencies (only needs to run once)
# You might have to restart your kernel after installation
!pip install requests sseclient-py pandas numpy

---
# 1. Connect to the Exchange

There are two exchanges:
- [**Test exchange**](http://ec2-52-49-69-152.eu-west-1.compute.amazonaws.com/) — for experimenting. 
- **Challenge exchange** — all trades count for final outcome. We will announce when account creation opens. **One account per team, one bot connection per account** — have one person create the team account and share the credentials.

Fill in your credentials below. We'll use the test exchange throughout this notebook — switch to the production URL when you're ready to compete.

In [ ]:
import json
import time
import requests
from pathlib import Path
from datetime import datetime, timezone

import pandas as pd
import numpy as np

from bot_template import BaseBot, OrderBook, Order, OrderRequest, OrderResponse, Trade, Side, Product

In [ ]:
TEST_URL = "http://ec2-52-49-69-152.eu-west-1.compute.amazonaws.com/"   # TODO: test exchange URL (use this for practice)
CHALLENGE_URL = "REPLACE_WITH_CHALLENGE_URL"   # TODO: challenge exchange URL (all trades on this exchange matter for the challenge!)

# We'll use the test exchange throughout this notebook.
# Switch to CHALLENGE_URL when you are ready to compete.
EXCHANGE_URL = TEST_URL

USERNAME = "REPLACE_WITH_NAME"  # TODO: your username
PASSWORD = "REPLACE_WITH_PASSWORD"  # TODO: your password

---
# 2. Discover Products

The exchange hosts 8 products across 3 real-world data sources plus 2 derived instruments. Each has:
- **symbol** — the product name you use in orders
- **tickSize** — minimum price increment (1 for all products)
- **contractSize** — multiplier for P&L (not relevant as we will normalize P&L in the end)

All products settle at **12pm London time** based on data observed during the 24-hour session (12pm to 12pm). **All settlement values are non-negative** — no product settles below zero.

The challenge exchange opens Saturday at 02:30pm and closes Sunday at 12:30pm .

## Thames Tidal Products

The Thames tidal level is measured at the Westminster gauge by the EA Flood Monitoring API.
Readings are in **metres Above Ordnance Datum (mAOD)** — the UK height reference where 0 mAOD equals mean sea level at Newlyn, Cornwall.

| Product | Settlement | Tick Size |
|---------|------------|------|
| **TIDE_SPOT** | Absolute value of tidal height at settlement time, in **mm AOD** (API returns metres — multiply by 1000) | 1 |
| **TIDE_SWING** | Sum of **strangle payoffs** on 15-min tidal absolute changes in cm (see below) | 1 |

### TIDE_SWING — the tidal strangle

A **strangle** is long put + long call at different strikes. It profits when the underlying moves **away from** the zone between the strikes.

For each consecutive 15-minute reading pair over the 24h session:
1. Compute `diff_cm = abs(level(t) − level(t−1)) × 100` (absolute change in centimetres)
2. Compute the strangle payoff with **strikes 20 and 25**:
   - Put leg: `max(0, 20 − diff_cm)`
   - Call leg: `max(0, diff_cm − 25)`
   - Total payoff = put + call (**zero** when diff is between 20–25 cm)
3. **Settlement = sum of all interval payoffs**

## Weather Products

London weather from [Open-Meteo](https://open-meteo.com/). Temperature in **Fahrenheit**, humidity in **%**.

| Product | Settlement | Tick Size |
|---------|------------|------|
| **WX_SPOT** | `temperature_F × humidity_%` at settlement time (12pm) | 1 |
| **WX_SUM** | `sum(temp_F × humidity_%)` at 15-min intervals over 24h, divided by 100 | 1 |

## Flight Products

Heathrow (LHR) flight data from [AeroDataBox](https://rapidapi.com/aedbx-aedbx/api/aerodatabox/).

| Product | Settlement | Tick Size |
|---------|------------|------|
| **LHR_COUNT** | Total number of flights (arrivals + departures) in the 24h session | 1 |
| **LHR_INDEX** | Imbalance metric: `abs(sum((arr − dep) / (arr + dep))) × 100` per 30-min interval | 1 |

## Derived Products

| Product | Settlement | Tick Size |
|---------|------------|------|
| **LON_ETF** | `TIDE_SPOT + WX_SPOT + LHR_COUNT` | 1 |
| **LON_FLY** | Options structure on ETF: `2×Put(6200) + Call(6200) − 2×Call(6600) + 3×Call(7000)` | 1 |

### LON_FLY — call butterfly + strangle

This decomposes into:
- **Call butterfly** (6200/6600/7000): `Call(6200) − 2×Call(6600) + Call(7000)` — peaks when ETF = 6600
- **2× Strangle** (Put@6200 / Call@7000): `2×Put(6200) + 2×Call(7000)` — unbounded profit on large moves

Combined: profits from large ETF moves in either direction, plus a bump near 6600.
At settlement, each option pays its **intrinsic value** (e.g. `Call(K) = max(0, ETF − K)`).

In [ ]:
class DummyBot(BaseBot):
    """Minimal bot for interactive use. """

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def on_orderbook(self, orderbook: OrderBook):
        pass  # we'll override this later

    def on_trades(self, trade: Trade):
        pass  # we'll override this later


bot = DummyBot(EXCHANGE_URL, USERNAME, PASSWORD)
print(f"Connected as '{USERNAME}'")

In [ ]:
products = bot.get_products()

print(f"{'Symbol':<15} {'Tick Size':>10} {'Start Price':>12} {'Contract':>10}")
print("-" * 50)
for p in products:
    print(f"{p.symbol:<15} {p.tickSize:>10} {p.startingPrice:>12} {p.contractSize:>10}")

---
# 3. Understanding the Orderbook

An `OrderBook` contains all resting buy and sell orders for a product.

- **buy_orders** — bids, sorted best-first (highest price first)
- **sell_orders** — asks, sorted best-first (lowest price first)

Each level has:
- `price` — the price level
- `volume` — total volume at that level (all participants)
- `own_volume` — how much of that volume is yours

The **spread** is the gap between the best bid and best ask. The **mid price** is the average of the two.

In [ ]:
# Fetch a live orderbook for the first product
product = products[0]
ob = bot.get_orderbook(product.symbol)

print(f"Orderbook for {ob.product} (tick_size={ob.tick_size})")
print()
print("  SELL (asks):")
for o in sorted(ob.sell_orders, key=lambda o: -o.price)[:5]:
    print(f"    {o.volume:>5} @ {o.price}")
print("  --------")
print("  BUY (bids):")
for o in sorted(ob.buy_orders, key=lambda o: -o.price)[:5]:
    print(f"    {o.volume:>5} @ {o.price}")

if ob.buy_orders and ob.sell_orders:
    mid = (ob.buy_orders[0].price + ob.sell_orders[0].price) / 2
    spread = ob.sell_orders[0].price - ob.buy_orders[0].price
    print(f"\n  Mid: {mid:.2f}  |  Spread: {spread:.2f}")

---
# 4. Trading

All orders are sent as `OrderRequest(product, price, side, volume)`.

Orders are of type **Good Till Cancel (GTC)** — they stay active on the book until filled or you cancel them.

## 4a. Place a resting order (GTC)

In [ ]:
# Place a buy order well below mid so it doesn't fill immediately
# Uncomment to run:

# order = OrderRequest(product=products[0].symbol, price=1.0, side=Side.BUY, volume=1)
# resp = bot.send_order(order)
# print(resp)

## 4b. Query active orders

See all your resting orders on the exchange.

In [ ]:
my_orders = bot.get_orders()
print(f"{len(my_orders)} active orders")
for o in my_orders:
    print(f"  {o['id'][:8]}  {o['side']:>4} {o['volume']}@{o['price']}  {o['product']}")

## 4c. Cancel orders

Cancel a specific order by ID, or cancel everything at once.

In [ ]:
# Cancel one order:
# bot.cancel_order("order-id-here")

# Cancel all your orders:
bot.cancel_all_orders()
print(f"Orders remaining: {len(bot.get_orders())}")

## 4d. Immediate-or-Cancel (IOC)

If you send an orer you might want to only engage as agressor and if your order does not get (fully) filled not end up in the order book with the remaining volume. Immediate-or-Cancel (IOC) orders are a type of order that achieves this by deleting all unfilled volume after one matching attempt. CMI does not support IOC orders natively but we can replicate them by **sending a GTC order and canceling any unfilled volume immdeiately**.

In [ ]:
def send_ioc(bot, order: OrderRequest) -> OrderResponse | None:
    """Send order, immediately cancel remainder. Simulates IOC."""
    resp = bot.send_order(order)
    if resp and resp.volume > 0:  # unfilled volume remains
        bot.cancel_order(resp.id)
    return resp


# Example: aggressively buy at the best ask price
# ob = bot.get_orderbook(products[0].symbol)
# if ob.sell_orders:
#     resp = send_ioc(bot, OrderRequest(ob.product, ob.sell_orders[0].price, Side.BUY, 1))
#     print(f"Filled: {resp.filled if resp else 0}")

## 4e. Positions

Check your position per product. Positive = long, negative = short.

In [ ]:
positions = bot.get_positions()
print("Positions:")
for product, pos in positions.items():
    print(f"  {product}: {pos:+d}")
if not positions:
    print("  (flat — no positions)")

---
# 5. Live Events
For your strategy you might want to react to certain market events which we support via **Server-Sent Events (SSE)**. The bot connects to a SSE stream with the exchange and triggers a callback on two types of events:

- **`on_orderbook(orderbook)`** — fires every time a product's orderbook changes (quotes being added/removed or a trade taking volume)
- **`on_trades(trade)`** — fires every time **you** trade

If you are building a quoter or hitter, these callbacks are where your strategy logic lives.
Please be aware that your own quotes, cancels and trades trigger on_orderbook and you are responsible for not spamming the exchange. Infinite update loops that put the exchange under sustaind load are not allowed and can result in a ban from this challenge. So please have safeguards in place to prevent your bot from going rogue. 

## 5a. Reacting to trades

In [ ]:
class TradeLogger(BaseBot):
    """Prints every trade as it happens."""

    def on_orderbook(self, orderbook: OrderBook):
        pass

    def on_trades(self, trade: Trade):
        print(f"{trade.product}  {trade.volume}@{trade.price}")


# Uncomment to run (interrupt to stop):
# Feel free to do some manual trades in the GUI to have them show up here
# logger = TradeLogger(EXCHANGE_URL, USERNAME, PASSWORD)
# logger.start()
# time.sleep(10)
# logger.stop()

## 5b. Reacting to orderbook changes

In [ ]:
class SpreadMonitor(BaseBot):
    """Prints the spread every time it changes."""

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self._last_spread: dict[str, float] = {}

    def on_orderbook(self, ob: OrderBook):
        if not ob.buy_orders or not ob.sell_orders:
            return
        spread = ob.sell_orders[0].price - ob.buy_orders[0].price
        if spread != self._last_spread.get(ob.product):
            mid = (ob.buy_orders[0].price + ob.sell_orders[0].price) / 2
            print(f"{ob.product}  mid={mid:.2f}  spread={spread:.2f}")
            self._last_spread[ob.product] = spread

    def on_trades(self, trade: Trade):
        pass


# Uncomment to run (interrupt to stop):
# monitor = SpreadMonitor(EXCHANGE_URL, USERNAME, PASSWORD)
# monitor.start()
# time.sleep(10)
# monitor.stop()

---
# 6. Strategy Ideas

Here are three broad approaches to automated trading. They can be combined.

## Market Making (Quoting)

Post **two-sided quotes** — a bid and an ask (buy and sell orders) — around your fair value estimate. You capture the **spread** (bid-ask difference) on every round-trip fill.

- P&L driver: spread capture on double sided fills
- Risk: adverse selection — the market moves through your quote before you can flatten

## Cross-Product Pricing (Arbitrage / Hitting)

Several products are related. Mispricings between them create **arbitrage** opportunities. When one product trades cheap relative to another, lift the cheap one and hit the expensive one simultaneously.

- P&L driver: capturing dislocations between correlated products
- Risk: leg risk — you get filled on one side but the other moves away before you can execute

## Alpha Research (Directional)

Use **external data** (weather, flights, tides) to build a view on where settlement will land. Take a directional position when your estimate diverges from the market price. Think about how to maximise your edge across timeframes — sitting at max position for the entire session means you can't capitalise on higher-frequency dislocations. Consider using your alpha signal to bias another strategy instead of just holding outright.

- P&L driver: correct predictions about settlement value
- Risk: your model is wrong, data arrives late, or the market already prices it in

---
# 7. Data Sources

Product settlements are driven by real-world data observed during the session window (12pm to 12pm London).
Below are helpers to fetch each data source.

## 7a. Weather — Open-Meteo (free, no API key)

15-minute resolution weather observations and forecasts for London via [Open-Meteo](https://open-meteo.com/).
Returns temperature (°C), wind speed, humidity (%), precipitation, cloud cover, visibility, and apparent temperature.

**Relevant for:** WX_SPOT (temp_F × humidity at 12pm), WX_SUM (15-min aggregate / 100).

In [ ]:
LONDON_LAT, LONDON_LON = 51.5074, -0.1278

def get_weather(past_steps=96, forecast_steps=96):
    """15-min weather for London. 96 steps = 24 hours.

    Returns DataFrame with: time, temperature, wind_speed, humidity,
    precipitation, cloud_cover, visibility, apparent_temperature.
    """
    variables = "temperature_2m,apparent_temperature,relative_humidity_2m,precipitation,wind_speed_10m,cloud_cover,visibility"
    resp = requests.get("https://api.open-meteo.com/v1/forecast", params={
        "latitude": LONDON_LAT, "longitude": LONDON_LON,
        "minutely_15": variables,
        "past_minutely_15": past_steps,
        "forecast_minutely_15": forecast_steps,
        "timezone": "Europe/London",
    })
    resp.raise_for_status()
    m = resp.json()["minutely_15"]
    return pd.DataFrame({
        "time": pd.to_datetime(m["time"]).tz_localize("Europe/London"),
        "temperature": m["temperature_2m"],
        "apparent_temperature": m["apparent_temperature"],
        "humidity": m["relative_humidity_2m"],
        "precipitation": m["precipitation"],
        "wind_speed": m["wind_speed_10m"],
        "cloud_cover": m["cloud_cover"],
        "visibility": m["visibility"],
    })

In [ ]:
df_weather = get_weather()
print(f"{len(df_weather)} readings, {df_weather.time.min()} -> {df_weather.time.max()}")
df_weather.tail(5)

## 7b. Thames Tidal Level — EA Flood Monitoring API (free, no API key)

Tidal level readings at the Westminster gauge, sampled every 15 minutes.
Levels are in **metres Above Ordnance Datum (mAOD)** — the UK height reference where 0 mAOD = mean sea level at Newlyn, Cornwall. Negative values mean the water surface is below the datum.

**Relevant for:** TIDE_SPOT (abs value in mm at 12pm), TIDE_SWING (strangle on 15-min diffs in cm).

In [ ]:
THAMES_MEASURE = "0006-level-tidal_level-i-15_min-mAOD"

def get_thames(limit=200):
    """Fetch recent Thames tidal readings at Westminster.

    Returns DataFrame with: time, level (mAOD).
    Use limit=400 for ~4 days of history.
    """
    resp = requests.get(
        f"https://environment.data.gov.uk/flood-monitoring/id/measures/{THAMES_MEASURE}/readings",
        params={"_sorted": "", "_limit": limit},
    )
    resp.raise_for_status()
    items = resp.json().get("items", [])
    df = pd.DataFrame(items)[["dateTime", "value"]].rename(columns={"dateTime": "time", "value": "level"})
    df["time"] = pd.to_datetime(df["time"], utc=True).dt.tz_convert("Europe/London")
    return df.sort_values("time").reset_index(drop=True)

In [ ]:
df_thames = get_thames(limit=200)
print(f"{len(df_thames)} readings, {df_thames.time.min()} -> {df_thames.time.max()}")
df_thames.tail(5)

## 7c. Flights — AeroDataBox via RapidAPI

Arrivals and departures at London Heathrow via the [AeroDataBox](https://rapidapi.com/aedbx-aedbx/api/aerodatabox/) flights endpoint. You'll need a free RapidAPI key — sign up [here](https://rapidapi.com/auth/sign-up) and subscribe to the Basic (free) tier. The free plan is limited to ~150 requests/month so be resourceful. Feel free to look for alternative endpoints, but settlement will be based on this data provider.

Two query styles are available (max 12h window each):
- **By relative time:** `offset_minutes` + `duration_minutes` relative to now
- **By time range:** explicit local times `fromLocal` / `toLocal` (format: `YYYY-MM-DDTHH:mm`)

The API also supports several boolean filters — check the [AeroDataBox docs](https://rapidapi.com/aedbx-aedbx/api/aerodatabox/playground/apiendpoint_3dbf8f9a-22de-4a99-8e7d-e542f6e63e4f) to learn what's available.

**Relevant for:** LHR_COUNT (total flights in 24h), LHR_INDEX (imbalance metric per 30-min interval).

In [ ]:
AERODATABOX_KEY = "YOUR_API_KEY"  # TODO: paste your RapidAPI key
AERODATABOX_HOST = "aerodatabox.p.rapidapi.com"
AIRPORT = "LHR"

def fetch_flights(airport=AIRPORT, offset_minutes=-360, duration_minutes=720, filters: dict | None = None):
    """Fetch flights by relative time window (offset from now).

    Args:
        airport: IATA code (default: LHR).
        offset_minutes: Start of window relative to now (negative = past).
        duration_minutes: Window length in minutes (max 720).
        filters: Optional dict of boolean query parameters to pass through
                 to the API (e.g. {"withLeg": True, "withLocation": False}).
    """
    params = f"?offsetMinutes={offset_minutes}&durationMinutes={duration_minutes}&direction=Both"
    if filters:
        for k, v in filters.items():
            params += f"&{k}={'true' if v else 'false'}"
    url = f"https://{AERODATABOX_HOST}/flights/airports/iata/{airport}{params}"
    resp = requests.get(url, headers={
        "x-rapidapi-host": AERODATABOX_HOST, "x-rapidapi-key": AERODATABOX_KEY})
    resp.raise_for_status()
    data = json.loads(resp.text)
    print(f"Fetched {len(data.get('arrivals', []))} arrivals, {len(data.get('departures', []))} departures")
    return data


def fetch_flights_range(airport=AIRPORT, from_local="2026-02-28T12:00", to_local="2026-02-29T00:00", filters: dict | None = None):
    """Fetch flights by explicit local time range (max 12h span).

    Args:
        airport: IATA code (default: LHR).
        from_local: Start time in local timezone (format: YYYY-MM-DDTHH:mm).
        to_local: End time in local timezone (format: YYYY-MM-DDTHH:mm).
        filters: Optional dict of boolean query parameters.
    """
    params = "?direction=Both"
    if filters:
        for k, v in filters.items():
            params += f"&{k}={'true' if v else 'false'}"
    url = f"https://{AERODATABOX_HOST}/flights/airports/iata/{airport}/{from_local}/{to_local}{params}"
    resp = requests.get(url, headers={
        "x-rapidapi-host": AERODATABOX_HOST, "x-rapidapi-key": AERODATABOX_KEY})
    resp.raise_for_status()
    data = json.loads(resp.text)
    print(f"Fetched {len(data.get('arrivals', []))} arrivals, {len(data.get('departures', []))} departures")
    return data

In [ ]:
# Fetch flights from the last 12 hours
data = fetch_flights(offset_minutes=-720, duration_minutes=720)
print(f"{len(data.get('arrivals', []))} arrivals, {len(data.get('departures', []))} departures")

# Look at one flight record to understand the structure
if data.get("arrivals"):
    print("\nExample arrival record:")
    print(json.dumps(data["arrivals"][0], indent=2))

---
# 8. Price-Time Priority

The exchange uses **price-time priority** for order matching:

1. **Price priority** — an incoming buy order fills against the **lowest-priced** sell order first. An incoming sell order fills against the **highest-priced** buy order first.
2. **Time priority** — at the same price level, the order that was placed **first** gets filled first.

This means if you and another participant both quote at the same price, whoever was there first gets the fill. If you cancel and re-place your order, you go to the **back of the queue** at that price level.

---
# 9. Simple Quoter

A minimal market-making loop: every N seconds, cancel all orders, compute mid, place a bid and ask at fixed width around mid.

In [ ]:
import math


class SimpleQuoter(BaseBot):
    """Cancel-and-replace quoter. Quotes fixed width around mid on all products."""

    def on_orderbook(self, ob):
        pass

    def on_trades(self, trade: Trade):
        side = "BOUGHT" if trade.buyer == self.username else "SOLD"
        print(f"  FILL: {side} {trade.volume}x {trade.product} @ {trade.price}")

    def run_loop(self, width=5.0, volume=5, interval=5):
        interval = max(interval, 1)
        products = {p.symbol: p for p in self.get_products()}
        self.start()  # start SSE so on_trades fires

        while True:
            self.cancel_all_orders()

            for symbol, product in products.items():
                ob = self.get_orderbook(symbol)

                # Mid = (best_bid + best_ask) / 2, ignoring our own orders
                bids = [o.price for o in ob.buy_orders if o.volume - o.own_volume > 0]
                asks = [o.price for o in ob.sell_orders if o.volume - o.own_volume > 0]
                mid = (max(bids) + min(asks)) / 2 if bids and asks else product.startingPrice

                tick = product.tickSize
                bid = math.floor((mid - width) / tick) * tick
                ask = math.ceil((mid + width) / tick) * tick

                if bid > 0 and bid < ask:
                    self.send_orders([
                        OrderRequest(symbol, bid, Side.BUY, volume),
                        OrderRequest(symbol, ask, Side.SELL, volume),
                    ])
                    print(f"  {symbol}  mid={mid:.0f}  {volume}@{bid} / {volume}@{ask}")

            time.sleep(interval)

In [ ]:
# Run the quoter — interrupt the cell to stop

quoter = SimpleQuoter(EXCHANGE_URL, USERNAME, PASSWORD)
print("Starting quoter (Ctrl+C to stop)...")
try:
    quoter.run_loop(width=5, volume=5, interval=5)
except KeyboardInterrupt:
    quoter.cancel_all_orders()
    quoter.stop()
    print("Quoter stopped.")

---
# Next Steps

This quoter is intentionally simple and missing functionality you might want to add. Here are a few ideas to improve it:

- **Use data sources** — adjust fair value based on weather, flights, or tides instead of only following mid (with a fallback to the starting price)
- **Adjust width** based on your confidence in your theo (your theoretical valuation)
- **Manage risk** — stop quoting one side when your position gets too large
- **Multiple levels** — quote at several price levels to provide more volume
- **Cross-product** — if products are related, use one to help price the other
- **Be smarter about cancels** — only requote when mid actually changes to preserve queue priority